In [1]:
import torch
import torch.nn as nn
torch.cuda.empty_cache()

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd

In [2]:
print(torch.__version__)

1.12.1


In [3]:
from sklearn.metrics import pairwise_distances

In [4]:
path_data_output='./data/ss_largeSizedDatasetRFAMonly/'

output_path_structureValidated_train_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_train_data.csv'
output_path_structureMFE_train_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_train_data.csv'

output_path_structureValidated_valid_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_valid_data.csv'
output_path_structureMFE_valid_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_valid_data.csv'

output_path_structureValidated_test_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structure_test_data.csv'
output_path_structureMFE_test_data=path_data_output+'ss_largeSizedDatasetRFAMonly_structureMFE_test_data.csv'

In [5]:
train_data_structureValidated=pd.read_csv(output_path_structureValidated_train_data,index_col=[0])
train_data_structureMFE=pd.read_csv(output_path_structureMFE_train_data,index_col=[0])

valid_data_structureValidated=pd.read_csv(output_path_structureValidated_valid_data,index_col=[0])
valid_data_structureMFE=pd.read_csv(output_path_structureMFE_valid_data,index_col=[0])

test_data_structureValidated=pd.read_csv(output_path_structureValidated_test_data,index_col=[0])
test_data_structureMFE=pd.read_csv(output_path_structureMFE_test_data,index_col=[0])

print(train_data_structureValidated.shape)
print(train_data_structureMFE.shape)
print(valid_data_structureValidated.shape)
print(valid_data_structureMFE.shape)
print(test_data_structureValidated.shape)
print(test_data_structureMFE.shape)

(11933, 17610)
(11933, 17610)
(149, 17610)
(149, 17610)
(430, 17610)
(430, 17610)


In [6]:
train_data_structureValidated_vectors=train_data_structureValidated.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)
train_data_structureMFE_vectors=train_data_structureMFE.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)

valid_data_structureValidated_vectors=valid_data_structureValidated.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)
valid_data_structureMFE_vectors=valid_data_structureMFE.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)

test_data_structureValidated_vectors=test_data_structureValidated.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)
test_data_structureMFE_vectors=test_data_structureMFE.drop(["family",'id_seq','is_subopt','bd_distance'],axis=1)

print(train_data_structureValidated_vectors.shape)
print(train_data_structureMFE_vectors.shape)
print(valid_data_structureValidated_vectors.shape)
print(valid_data_structureMFE_vectors.shape)
print(test_data_structureValidated_vectors.shape)
print(test_data_structureMFE_vectors.shape)

(11933, 17606)
(11933, 17606)
(149, 17606)
(149, 17606)
(430, 17606)
(430, 17606)


In [7]:
# Assuming you have your dataset loaded into pandas DataFrames: train_data, test_data

# Convert pandas DataFrames to numpy arrays
train_structure_validated_np = train_data_structureValidated_vectors.to_numpy()
train_structure_mfe_np = train_data_structureMFE_vectors.to_numpy()

valid_structure_validated_np = valid_data_structureValidated_vectors.to_numpy()
valid_structure_mfe_np = valid_data_structureMFE_vectors.to_numpy()

test_structure_validated_np = test_data_structureValidated_vectors.to_numpy()
test_structure_mfe_np = test_data_structureMFE_vectors.to_numpy()

# Convert back to pandas DataFrames if needed
#train_noisy_data = pd.DataFrame(train_noisy_np, columns=train_data.columns)
#test_noisy_data = pd.DataFrame(test_noisy_np, columns=test_data.columns)

# Convert numpy arrays to PyTorch tensors
train_structure_validated_tensor = torch.tensor(train_structure_validated_np, dtype=torch.float32)
train_structure_mfe_tensor = torch.tensor(train_structure_mfe_np, dtype=torch.float32)

valid_structure_validated_tensor = torch.tensor(valid_structure_validated_np, dtype=torch.float32)
valid_structure_mfe_tensor = torch.tensor(valid_structure_mfe_np, dtype=torch.float32)

test_structure_validated_tensor = torch.tensor(test_structure_validated_np, dtype=torch.float32)
test_structure_mfe_tensor = torch.tensor(test_structure_mfe_np, dtype=torch.float32)

# Create DataLoader for batch processing

#batch_size = 1000

batch_size = 164
#batch_size = 32

train_structure_loader = DataLoader(TensorDataset(train_structure_mfe_tensor, train_structure_validated_tensor), batch_size=batch_size, shuffle=True)
valid_structure_loader = DataLoader(TensorDataset(valid_structure_mfe_tensor, valid_structure_validated_tensor), batch_size=batch_size, shuffle=True)
test_structure_loader = DataLoader(TensorDataset(test_structure_mfe_tensor, test_structure_validated_tensor), batch_size=batch_size, shuffle=False)


# train_structure_validated_loader = DataLoader(TensorDataset(train_structure_validated_tensor, train_structure_validated_tensor), batch_size=batch_size, shuffle=True)
# train_structure_mfe_loader = DataLoader(TensorDataset(train_structure_mfe_tensor, train_structure_mfe_tensor), batch_size=batch_size, shuffle=True)

# test_structure_74validated_loader = DataLoader(TensorDataset(test_structure_validated_tensor, test_structure_validated_tensor), batch_size=batch_size, shuffle=False)
# test_structure_mfe_loader = DataLoader(TensorDataset(test_structure_mfe_tensor, test_structure_mfe_tensor), batch_size=batch_size, shuffle=False)


In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [9]:
# Define the denoising autoencoder architecture
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_size):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size,2000 ),
             nn.ReLU(),
            nn.Linear( 2000,500),
             nn.ReLU(),
             nn.Linear( 500,200),
             nn.ReLU(),
            nn.Linear(200, 50),
            )
        self.decoder = nn.Sequential(
             nn.Linear(50, 200),
            nn.ReLU(),
            nn.Linear(200, 500),
            nn.ReLU(),
            nn.Linear(500, 2000),
            nn.ReLU(),
            nn.Linear( 2000,input_size),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Instantiate the model
input_size = len(train_data_structureValidated_vectors.columns)  # Number of features
#hidden_size = 64
model = DenoisingAutoencoder(input_size).to(device)

print(model)

# Define loss function and optimizer
criterion = nn.MSELoss()
#criterion = nn.L1Loss()

optimizer = optim.Adam(model.parameters(), lr=0.01)

DenoisingAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=17606, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=50, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=2000, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2000, out_features=17606, bias=True)
  )
)


In [14]:
# Training the model
num_epochs = 5000

list_avg_loss_train=[]
list_avg_loss_valid=[]

for epoch in range(num_epochs):
    
    valid_loss = 0.0
    min_delta=0.1

    if len(list_avg_loss_valid)>0 and valid_loss>(np.min(list_avg_loss_valid)+min_delta) :
        print('early stopping:' +str(valid_loss) +' > min ' +str(np.min(list_avg_loss_valid)))
        break;
    
    #Train
    running_loss = 0.0
     
    for data in train_structure_loader:

        inputs_structure_mfe_, inputs_structure_validated_ = data
        inputs_structure_mfe, inputs_structure_validated = inputs_structure_mfe_.to(device), inputs_structure_validated_.to(device)
        
        outputs = model(inputs_structure_mfe )
        loss = criterion(outputs,inputs_structure_mfe  )
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
                
        #print(running_loss)
    #print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_structure_loader)}")
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss}")

    
    running_cos_distance_outputs_vs_structure_validated=0
    running_cos_distance_mfe_vs_structure_validated=0
    
    #running_cos_distance_struct_mfe_vs_structure_validated=0
    #running_cos_distance_struct_outputs_vs_structure_validated=0
    
    running_cos_distance_mfe_vs_structure_validated_sd=0
    running_cos_distance_outputs_vs_structure_validated_sd=0
    
    #Valid
    with torch.no_grad():
        
        for data in valid_structure_loader:
            
            inputs_valid_structure_mfe_, inputs_valid_structure_validated_ = data
            inputs_valid_structure_mfe, inputs_valid_structure_validated = inputs_valid_structure_mfe_.to(device), inputs_valid_structure_validated_.to(device)
        
            outputs_valid = model(inputs_valid_structure_mfe)
            valid_loss += criterion(outputs_valid, inputs_valid_structure_validated).item()

            running_cos_distance_mfe_vs_structure_validated+=np.mean(pairwise_distances(inputs_valid_structure_mfe.cpu().numpy(), inputs_valid_structure_validated.cpu().numpy(), metric='cosine').diagonal())
            running_cos_distance_outputs_vs_structure_validated+=np.mean(pairwise_distances(outputs_valid.cpu().numpy(), inputs_valid_structure_validated.cpu().numpy(), metric='cosine').diagonal())

            running_cos_distance_mfe_vs_structure_validated_sd+=np.std(pairwise_distances(inputs_valid_structure_mfe.cpu().numpy(), inputs_valid_structure_validated.cpu().numpy(), metric='euclidean').diagonal())
            running_cos_distance_outputs_vs_structure_validated_sd+=np.std(pairwise_distances(outputs_valid.cpu().numpy(), inputs_valid_structure_validated.cpu().numpy(), metric='euclidean').diagonal())

            #running_cos_distance_struct_mfe_vs_structure_validated+=np.median(pairwise_distances(inputs_valid_structure_mfe.cpu().numpy()[:,0:4332]  , inputs_valid_structure_validated.cpu().numpy()[:,0:4332]  , metric='cosine').diagonal())
            #running_cos_distance_struct_outputs_vs_structure_validated+=np.median(pairwise_distances(outputs_valid.cpu().numpy()[:,0:4332]  , inputs_valid_structure_validated.cpu().numpy()[:,0:4332]  , metric='cosine').diagonal())
        
    #print(f"Valid Loss: {valid_loss / len(valid_structure_loader)}")
    print(f"Valid Loss: {valid_loss}")
    print(f"cos_mfe_vs_struc_valid: {running_cos_distance_mfe_vs_structure_validated}, cos_output_vs_struc_valid: {running_cos_distance_outputs_vs_structure_validated}")
    print(f"cos_output_vs_struc_valid_sd: {running_cos_distance_mfe_vs_structure_validated_sd}, cos_output_vs_struc_valid_sd: {running_cos_distance_outputs_vs_structure_validated_sd}")

    #print(f"cos_struct_mfe_vs_struc_valid: {running_cos_distance_struct_mfe_vs_structure_validated}, cos_struct_output_vs_struc_valid: {running_cos_distance_struct_outputs_vs_structure_validated}")

    list_avg_loss_valid.append(valid_loss)
    
    

Epoch [1/5000], Loss: 0.14215625612996519
Valid Loss: 0.0069952611811459064
cos_mfe_vs_struc_valid: 0.09965310245752335, cos_output_vs_struc_valid: 0.1312338262796402
cos_output_vs_struc_valid_sd: 3.957202434539795, cos_output_vs_struc_valid_sd: 2.450716972351074
Epoch [2/5000], Loss: 0.13294253533240408
Valid Loss: 0.00674877455458045
cos_mfe_vs_struc_valid: 0.09965308755636215, cos_output_vs_struc_valid: 0.12807710468769073
cos_output_vs_struc_valid_sd: 3.957202434539795, cos_output_vs_struc_valid_sd: 2.3842759132385254
Epoch [3/5000], Loss: 0.12938790582120419
Valid Loss: 0.006801909301429987
cos_mfe_vs_struc_valid: 0.09965310245752335, cos_output_vs_struc_valid: 0.1275213211774826
cos_output_vs_struc_valid_sd: 3.957202434539795, cos_output_vs_struc_valid_sd: 2.394320487976074
Epoch [4/5000], Loss: 0.12713309261016548
Valid Loss: 0.006764768622815609
cos_mfe_vs_struc_valid: 0.09965308755636215, cos_output_vs_struc_valid: 0.12658028304576874
cos_output_vs_struc_valid_sd: 3.9572024345

KeyboardInterrupt: 

In [ ]:
# Evaluating the model
model.eval()
test_loss = 0.0

with torch.no_grad():
    for data in test_structure_loader:
        inputs_structure_mfe_, inputs_structure_validated_ = data
        inputs_structure_mfe, inputs_structure_validated = inputs_structure_mfe_.to(device), inputs_structure_validated_.to(device)

        outputs = model(inputs_structure_mfe)
        test_loss += criterion(outputs, inputs_structure_validated).item()
        test_loss += criterion(outputs, inputs_structure_validated).item()

#print(f"Test Loss: {test_loss / len(test_structure_loader)}")
print(f"Test Loss: {test_loss }")

In [ ]:
with torch.no_grad():
    outputs_ = model(test_structure_mfe_tensor.to(device))
    
    outputs=outputs_.cpu()

In [ ]:
print(outputs.numpy().shape)
outputs.numpy()

In [ ]:
print(test_structure_mfe_tensor.numpy().shape)
test_structure_mfe_tensor.numpy()

In [ ]:
print(test_structure_validated_tensor.numpy().shape)
test_structure_validated_tensor.numpy()

In [ ]:
cos_validated_vs_mfe=pairwise_distances(test_structure_validated_tensor.numpy()[:,0:4332] , test_structure_mfe_tensor.numpy()[:,0:4332] , metric='cosine').diagonal()
cos_validated_vs_denoised_mfe=pairwise_distances(test_structure_validated_tensor.numpy()[:,0:4332] , outputs.numpy()[:,0:4332] , metric='cosine').diagonal()

pd.DataFrame({'cos_validated_vs_mfe':cos_validated_vs_mfe,'cos_validated_vs_denoised_mfe':cos_validated_vs_denoised_mfe})

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
 
#Creating dataset
 
data = [cos_validated_vs_mfe, cos_validated_vs_denoised_mfe]
 
fig = plt.figure(figsize =(10, 7))
 
# Creating axes instance
ax = fig.add_axes([0, 0, 1,1])
 
# Creating plot
bp = ax.boxplot(data)
 
# show plot
plt.show()